In [1]:
import os
%pwd

'f:\\ml projects\\brain_tumor_classification\\notebooks'

In [2]:
os.chdir('../')
%pwd

'f:\\ml projects\\brain_tumor_classification'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir:Path
    tensorboard_log_dir:Path
    checkpoint_model_path: Path

In [4]:
from brain_tumor_classification.constants import *
from brain_tumor_classification.utils.common import create_directories,read_yaml

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILEPATH,params_filepath = PARAMS_FILEPATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        temp_config = self.config.prepare_callbacks

        create_directories([temp_config.root_dir])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = temp_config.root_dir,
            tensorboard_log_dir = temp_config.tensorboard_log_dir,
            checkpoint_model_path = temp_config.checkpoint_model_path
        )

        return prepare_callback_config

In [5]:
import tensorflow as tf
from datetime import datetime

In [10]:
class PrepareCallback:
    def __init__(self,config:PrepareCallbackConfig) -> None:
        self.config = config

    @property
    def _create_tensorboard(self):
        log_dir_name = f"TB_LOG_at{datetime.now().strftime('%d_%m_%y_%H_%M_%S')}"
        tensorboard_log_dir = os.path.join(self.config.tensorboard_log_dir,log_dir_name)

        return tf.keras.callbacks.TensorBoard(log_dir = tensorboard_log_dir)

    @property
    def _create_model_callback(self):
        return tf.keras.callbacks.ModelCheckpoint(self.config.checkpoint_model_path,save_best_only = True)
    
    def create_TB_CP(self):
        return [self._create_tensorboard,self._create_model_callback]
    

In [11]:
try:
    config = ConfigurationManager()
    get_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config = get_callbacks_config)
    callbacks_list =  prepare_callbacks.create_TB_CP()
except Exception as e:
    raise e